In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import librosa, librosa.display
import os
import tensorflow as tf
print(tf.__version__)
import re
import sklearn
import datetime
import pandas as apd
import scipy, matplotlib.pyplot as plt, IPython.display as ipd
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, balanced_accuracy_score
%load_ext tensorboard
tf.random.set_seed(999)
np.random.seed(999)
from tensorflow.keras.layers import Conv2D, Input, GlobalAveragePooling2D, LeakyReLU, Flatten, Dense, Reshape, MaxPooling2D, Conv2DTranspose, BatchNormalization, Activation, Input, Dropout, SpatialDropout2D
from tensorflow.keras import Model, Sequential

2.0.0


In [2]:
## librosa config
NUM_CLASSES=5
SAMPLE_RATE=16000
N_MFCC=40
WINDOW_LENGTH=512
OVERLAP=round(0.25 * WINDOW_LENGTH)
BATCH_SIZE=8
LEARNING_RATE=8e-5

In [3]:
# save the numpy data
X_train = np.load('../data/processed/TDC_VoIP_train_STFT_SPEC.npy', allow_pickle=True).item()
X_test = np.load('../data/processed/TDC_VoIP_test_STFT_SPEC.npy', allow_pickle=True).item()

In [4]:
train_filenames = X_train['filenames']
y_train_scores = X_train['sample_MOS']
y_train_classes = X_train['class_labels']
X_train_features = X_train['features']
input_height = X_train_features.shape[1]

print("Train data")
print("Features shape:", X_train_features.shape)
print("class_labels shape:", y_train_classes.shape)
print("sample_MOS shape:", y_train_scores.shape)

Train data
Features shape: (327, 257, 1241, 1)
class_labels shape: (327,)
sample_MOS shape: (327,)


In [5]:
test_filenames = X_test['filenames']
y_test_scores = X_test['sample_MOS']
y_test_classes = X_test['class_labels']
X_test_features = X_test['features']

print("Test data")
print("Features shape:", X_test_features.shape)
print("class_labels shape:", y_test_classes.shape)
print("sample_MOS shape:", y_test_scores.shape)

Test data
Features shape: (57, 257, 1241, 1)
class_labels shape: (57,)
sample_MOS shape: (57,)


In [6]:

## normalization
def whitening(feature_vector):
    # print(feature_vector.shape)
    mean = np.expand_dims(np.mean(feature_vector))
    std = np.expand_dims(np.std(feature_vector))
    feature_vector = (feature_vector - mean) / std
    return feature_vector

## ConvNet architecture

In [7]:
def build_model(input_shape):
    inputs = Input(shape=input_shape)
    x = inputs

    # Conv --> BN --> Activation --> MaxPool --> [Dropout 2D]
    x = Conv2D(filters=16, kernel_size=3, padding='same', input_shape=input_shape)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling2D(pool_size=2)(x)

    x = Conv2D(filters=24, kernel_size=3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling2D(pool_size=2)(x)

    x = Conv2D(filters=24, kernel_size=3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling2D(pool_size=2)(x)
    x = SpatialDropout2D(0.15)(x)

    x = Conv2D(filters=32, kernel_size=3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling2D(pool_size=2)(x)
    x = SpatialDropout2D(0.25)(x)
    
    # scores branch 
    x_score = Conv2D(filters=32, kernel_size=3, padding='same')(x)
    x_score = BatchNormalization()(x_score)
    x_score = Activation('relu')(x_score)
    x_score = MaxPooling2D(pool_size=2)(x_score)
    x_score = SpatialDropout2D(0.25)(x_score)

    x_score = Conv2D(filters=32, kernel_size=3, padding='same')(x_score)
    x_score = BatchNormalization()(x_score)
    x_score = Activation('relu')(x_score)
    x_score = MaxPooling2D(pool_size=2)(x_score)
    x_score = SpatialDropout2D(0.25)(x_score)
    x_score = GlobalAveragePooling2D()(x_score)
    x_score = Dense(1, name='output_mos')(x_score)

#     # claas branch 
#     x_class = Conv2D(filters=32, kernel_size=3, padding='same')(x)
#     x_class = BatchNormalization()(x_class)
#     x_class = Activation('relu')(x_class)
#     x_class = MaxPooling2D(pool_size=2)(x_class)
#     x_class = SpatialDropout2D(0.25)(x_class)

#     x_class = Conv2D(filters=48, kernel_size=3, padding='same')(x_class)
#     x_class = BatchNormalization()(x_class)
#     x_class = Activation('relu')(x_class)
#     x_class = MaxPooling2D(pool_size=2)(x_class)
#     x_class = SpatialDropout2D(0.35)(x_class)
#     x_class = GlobalAveragePooling2D()(x_class)
#     x_class = Dense(NUM_CLASSES, activation='softmax', name='output_class')(x_class)
    
    model = Model(inputs=inputs, outputs=x_score)
    return model

In [8]:
model_SPEC = build_model((input_height, None, 1))

optimizer = tf.keras.optimizers.Adam(LEARNING_RATE)
model_SPEC.compile(loss={'output_mos': 'mse'}, 
              loss_weights={'output_mos': 1},
              metrics={'output_mos': tf.keras.metrics.MeanAbsoluteError()},
              optimizer=optimizer)

# Display model architecture summary 
model_SPEC.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 257, None, 1)]    0         
_________________________________________________________________
conv2d (Conv2D)              (None, 257, None, 16)     160       
_________________________________________________________________
batch_normalization (BatchNo (None, 257, None, 16)     64        
_________________________________________________________________
activation (Activation)      (None, 257, None, 16)     0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 128, None, 16)     0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 128, None, 24)     3480      
_________________________________________________________________
batch_normalization_1 (Batch (None, 128, None, 24)     96    

In [9]:
monitor_metric='val_loss'

early_stopping_callback = tf.keras.callbacks.EarlyStopping(mode='auto', monitor=monitor_metric, patience=100, restore_best_weights=True)

# logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
# tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, update_freq='epoch')
#checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath='drive/My Drive/audio-quality-cnn/model.h5', 
#                                                          monitor=monitor_metric, save_best_only=True)

In [10]:
def whitening(image, y1, y2):
    # normalization using the image mean and std tends to produce under-optimized models
    # mean = tf.math.reduce_mean(image)
    # std = tf.math.reduce_std(image)
    mean = tf.expand_dims(tf.math.reduce_mean(image, axis=1), axis=1)
    std = tf.expand_dims(tf.math.reduce_std(image, axis=1), axis=1)
    image = (image - mean) / std
    return image, y1, y2
    
def random_crop(image, y1, y2):
    h = image.shape[0]
    return tf.math.abs(tf.image.random_crop(image, size=(h,416,1))), y1, y2

def cutout(image, y1, y2, cutout_size=24):
    h, w = image.shape[:2]
    mask = tf.zeros((24,24))

    left = tf.random.uniform(shape=(), minval=0, maxval=w-cutout_size, dtype=tf.int32)
    right = w - (cutout_size + left)

    top = tf.random.uniform(shape=(), minval=0, maxval=h-cutout_size, dtype=tf.int32)
    bottom = h - (cutout_size + top)

    mask = tf.pad(mask, [[top, bottom], [left, right]], constant_values=1)
    image = image * tf.expand_dims(mask, 2)
    return image, label

def prepare_input_format(image, y1, y2):
    return image, {'output_mos': y1, 'output_class': y2}

In [11]:
train_dataset = tf.data.Dataset.from_tensor_slices((X_train_features, y_train_scores, y_train_classes))
train_dataset = train_dataset.map(random_crop)
train_dataset = train_dataset.map(whitening)
train_dataset = train_dataset.map(prepare_input_format)
train_dataset = train_dataset.shuffle(buffer_size=1024)
train_dataset = train_dataset.repeat()
train_dataset = train_dataset.batch(BATCH_SIZE)

In [12]:
test_dataset = tf.data.Dataset.from_tensor_slices((X_test_features, y_test_scores, y_test_classes))
test_dataset = test_dataset.map(whitening)
test_dataset = test_dataset.map(prepare_input_format)
test_dataset = test_dataset.repeat(1)
test_dataset = test_dataset.batch(256)

In [13]:
steps_per_epoch = X_train_features.shape[0] // BATCH_SIZE
print("steps_per_epoch:",steps_per_epoch)

steps_per_epoch: 40


In [14]:
history_SPEC = model_SPEC.fit(X_train_features,
          {'output_mos': y_train_scores},
          validation_data=(X_test_features, 
                           {'output_mos': y_test_scores}),
          batch_size=BATCH_SIZE,
          epochs=1000,
          callbacks=[early_stopping_callback]
        )

Train on 327 samples, validate on 57 samples
Epoch 1/1000
327/327 [==============================] - 108s 330ms/sample - loss: 10.6165 - mean_absolute_error: 3.0397 - val_loss: 9.7536 - val_mean_absolute_error: 2.8245
Epoch 2/1000
327/327 [==============================] - 119s 363ms/sample - loss: 8.7753 - mean_absolute_error: 2.7320 - val_loss: 11.1360 - val_mean_absolute_error: 2.9179
Epoch 3/1000
327/327 [==============================] - 151s 461ms/sample - loss: 7.5786 - mean_absolute_error: 2.5137 - val_loss: 11.8985 - val_mean_absolute_error: 2.9984
Epoch 4/1000
327/327 [==============================] - 162s 495ms/sample - loss: 6.8834 - mean_absolute_error: 2.3629 - val_loss: 12.1636 - val_mean_absolute_error: 3.0400
Epoch 5/1000
327/327 [==============================] - 144s 439ms/sample - loss: 6.2265 - mean_absolute_error: 2.2128 - val_loss: 12.9919 - val_mean_absolute_error: 3.0776
Epoch 6/1000
327/327 [==============================] - 131s 401ms/sample - loss: 5.7701 -

327/327 [==============================] - 175s 535ms/sample - loss: 1.8460 - mean_absolute_error: 1.1214 - val_loss: 3.8589 - val_mean_absolute_error: 1.7129
Epoch 49/1000
327/327 [==============================] - 186s 567ms/sample - loss: 1.5815 - mean_absolute_error: 1.0273 - val_loss: 3.8027 - val_mean_absolute_error: 1.6949
Epoch 50/1000
327/327 [==============================] - 186s 569ms/sample - loss: 1.6632 - mean_absolute_error: 1.0767 - val_loss: 4.1065 - val_mean_absolute_error: 1.7688
Epoch 51/1000
327/327 [==============================] - 192s 588ms/sample - loss: 1.6867 - mean_absolute_error: 1.0948 - val_loss: 3.5753 - val_mean_absolute_error: 1.6325
Epoch 52/1000
327/327 [==============================] - 202s 618ms/sample - loss: 1.5863 - mean_absolute_error: 1.0358 - val_loss: 3.2796 - val_mean_absolute_error: 1.5498
Epoch 53/1000
327/327 [==============================] - 191s 585ms/sample - loss: 1.6669 - mean_absolute_error: 1.0557 - val_loss: 3.3816 - val_mean

327/327 [==============================] - 117s 357ms/sample - loss: 1.1867 - mean_absolute_error: 0.8927 - val_loss: 1.7631 - val_mean_absolute_error: 1.1460
Epoch 96/1000
327/327 [==============================] - 117s 358ms/sample - loss: 1.0697 - mean_absolute_error: 0.8407 - val_loss: 1.8405 - val_mean_absolute_error: 1.1743
Epoch 97/1000
327/327 [==============================] - 121s 369ms/sample - loss: 1.1598 - mean_absolute_error: 0.9009 - val_loss: 1.8098 - val_mean_absolute_error: 1.1625
Epoch 98/1000
327/327 [==============================] - 118s 361ms/sample - loss: 1.2356 - mean_absolute_error: 0.9220 - val_loss: 1.9319 - val_mean_absolute_error: 1.2050
Epoch 99/1000
327/327 [==============================] - 120s 366ms/sample - loss: 1.0203 - mean_absolute_error: 0.8503 - val_loss: 1.8200 - val_mean_absolute_error: 1.1659
Epoch 100/1000
327/327 [==============================] - 124s 379ms/sample - loss: 1.1819 - mean_absolute_error: 0.8816 - val_loss: 1.9274 - val_mea

327/327 [==============================] - 113s 346ms/sample - loss: 0.9836 - mean_absolute_error: 0.8040 - val_loss: 1.3737 - val_mean_absolute_error: 1.0170
Epoch 143/1000
327/327 [==============================] - 113s 346ms/sample - loss: 1.0287 - mean_absolute_error: 0.8223 - val_loss: 1.3410 - val_mean_absolute_error: 1.0015
Epoch 144/1000
327/327 [==============================] - 113s 346ms/sample - loss: 0.9624 - mean_absolute_error: 0.8167 - val_loss: 1.3286 - val_mean_absolute_error: 0.9975
Epoch 145/1000
327/327 [==============================] - 113s 347ms/sample - loss: 0.9822 - mean_absolute_error: 0.8094 - val_loss: 1.3273 - val_mean_absolute_error: 0.9965
Epoch 146/1000
327/327 [==============================] - 113s 346ms/sample - loss: 0.8834 - mean_absolute_error: 0.7731 - val_loss: 1.4384 - val_mean_absolute_error: 1.0436
Epoch 147/1000
327/327 [==============================] - 114s 347ms/sample - loss: 0.9280 - mean_absolute_error: 0.7864 - val_loss: 1.3019 - val

327/327 [==============================] - 113s 346ms/sample - loss: 1.0169 - mean_absolute_error: 0.8240 - val_loss: 1.1652 - val_mean_absolute_error: 0.9072
Epoch 190/1000
327/327 [==============================] - 113s 347ms/sample - loss: 0.8539 - mean_absolute_error: 0.7578 - val_loss: 1.1844 - val_mean_absolute_error: 0.9203
Epoch 191/1000
327/327 [==============================] - 120s 366ms/sample - loss: 0.8301 - mean_absolute_error: 0.7449 - val_loss: 1.3473 - val_mean_absolute_error: 1.0007
Epoch 192/1000
327/327 [==============================] - 132s 405ms/sample - loss: 0.8426 - mean_absolute_error: 0.7465 - val_loss: 1.3217 - val_mean_absolute_error: 0.9895
Epoch 193/1000
327/327 [==============================] - 116s 355ms/sample - loss: 0.8419 - mean_absolute_error: 0.7501 - val_loss: 1.2174 - val_mean_absolute_error: 0.9404
Epoch 194/1000
327/327 [==============================] - 113s 346ms/sample - loss: 0.9799 - mean_absolute_error: 0.8201 - val_loss: 1.2682 - val

...

327/327 [==============================] - 113s 345ms/sample - loss: 0.4856 - mean_absolute_error: 0.5437 - val_loss: 0.9154 - val_mean_absolute_error: 0.7912
Epoch 472/1000
327/327 [==============================] - 112s 343ms/sample - loss: 0.5221 - mean_absolute_error: 0.5839 - val_loss: 0.9668 - val_mean_absolute_error: 0.8068
Epoch 473/1000
327/327 [==============================] - 112s 344ms/sample - loss: 0.5839 - mean_absolute_error: 0.6218 - val_loss: 1.0092 - val_mean_absolute_error: 0.8550
Epoch 474/1000
327/327 [==============================] - 113s 347ms/sample - loss: 0.5117 - mean_absolute_error: 0.5609 - val_loss: 0.8834 - val_mean_absolute_error: 0.7786
Epoch 475/1000
327/327 [==============================] - 112s 343ms/sample - loss: 0.4673 - mean_absolute_error: 0.5403 - val_loss: 0.8998 - val_mean_absolute_error: 0.7838
Epoch 476/1000
327/327 [==============================] - 113s 345ms/sample - loss: 0.5103 - mean_absolute_error: 0.5580 - val_loss: 0.9128 - val

327/327 [==============================] - 113s 344ms/sample - loss: 0.4311 - mean_absolute_error: 0.5230 - val_loss: 0.8988 - val_mean_absolute_error: 0.7837
Epoch 519/1000
327/327 [==============================] - 114s 349ms/sample - loss: 0.4623 - mean_absolute_error: 0.5405 - val_loss: 0.8254 - val_mean_absolute_error: 0.7536
Epoch 520/1000
327/327 [==============================] - 112s 343ms/sample - loss: 0.4596 - mean_absolute_error: 0.5530 - val_loss: 0.8142 - val_mean_absolute_error: 0.7507
Epoch 521/1000
327/327 [==============================] - 113s 344ms/sample - loss: 0.5193 - mean_absolute_error: 0.5746 - val_loss: 0.8136 - val_mean_absolute_error: 0.7507
Epoch 522/1000
327/327 [==============================] - 113s 345ms/sample - loss: 0.4667 - mean_absolute_error: 0.5440 - val_loss: 0.8630 - val_mean_absolute_error: 0.7706
Epoch 523/1000
327/327 [==============================] - 113s 344ms/sample - loss: 0.3998 - mean_absolute_error: 0.4994 - val_loss: 0.9377 - val

327/327 [==============================] - 113s 344ms/sample - loss: 0.4769 - mean_absolute_error: 0.5464 - val_loss: 0.8382 - val_mean_absolute_error: 0.7504
Epoch 566/1000
327/327 [==============================] - 112s 344ms/sample - loss: 0.4613 - mean_absolute_error: 0.5380 - val_loss: 1.1095 - val_mean_absolute_error: 0.8658
Epoch 567/1000
327/327 [==============================] - 113s 345ms/sample - loss: 0.4806 - mean_absolute_error: 0.5450 - val_loss: 0.9668 - val_mean_absolute_error: 0.8132
Epoch 568/1000
327/327 [==============================] - 113s 345ms/sample - loss: 0.3888 - mean_absolute_error: 0.5028 - val_loss: 0.8739 - val_mean_absolute_error: 0.7741
Epoch 569/1000
327/327 [==============================] - 113s 344ms/sample - loss: 0.4088 - mean_absolute_error: 0.5139 - val_loss: 0.8184 - val_mean_absolute_error: 0.7513
Epoch 570/1000
327/327 [==============================] - 113s 344ms/sample - loss: 0.4123 - mean_absolute_error: 0.4955 - val_loss: 0.8736 - val

327/327 [==============================] - 112s 344ms/sample - loss: 0.4076 - mean_absolute_error: 0.5114 - val_loss: 0.7749 - val_mean_absolute_error: 0.7277
Epoch 613/1000
327/327 [==============================] - 112s 343ms/sample - loss: 0.4091 - mean_absolute_error: 0.5098 - val_loss: 1.1983 - val_mean_absolute_error: 0.8823
Epoch 614/1000
327/327 [==============================] - 112s 343ms/sample - loss: 0.4212 - mean_absolute_error: 0.5101 - val_loss: 0.9250 - val_mean_absolute_error: 0.7905
Epoch 615/1000
327/327 [==============================] - 112s 344ms/sample - loss: 0.3625 - mean_absolute_error: 0.4793 - val_loss: 0.9339 - val_mean_absolute_error: 0.7966
Epoch 616/1000
327/327 [==============================] - 112s 343ms/sample - loss: 0.4948 - mean_absolute_error: 0.5520 - val_loss: 0.7605 - val_mean_absolute_error: 0.7258
Epoch 617/1000
327/327 [==============================] - 112s 343ms/sample - loss: 0.4148 - mean_absolute_error: 0.4977 - val_loss: 1.0798 - val

327/327 [==============================] - 112s 343ms/sample - loss: 0.3986 - mean_absolute_error: 0.5104 - val_loss: 0.9033 - val_mean_absolute_error: 0.7860
Epoch 660/1000
327/327 [==============================] - 113s 345ms/sample - loss: 0.3866 - mean_absolute_error: 0.4916 - val_loss: 1.1147 - val_mean_absolute_error: 0.8665
Epoch 661/1000
327/327 [==============================] - 113s 347ms/sample - loss: 0.3727 - mean_absolute_error: 0.4881 - val_loss: 0.9108 - val_mean_absolute_error: 0.7887
Epoch 662/1000
327/327 [==============================] - 112s 343ms/sample - loss: 0.4146 - mean_absolute_error: 0.5068 - val_loss: 0.8529 - val_mean_absolute_error: 0.7631
Epoch 663/1000
327/327 [==============================] - 113s 347ms/sample - loss: 0.3616 - mean_absolute_error: 0.4684 - val_loss: 0.9182 - val_mean_absolute_error: 0.7980
Epoch 664/1000
327/327 [==============================] - 112s 343ms/sample - loss: 0.3645 - mean_absolute_error: 0.4759 - val_loss: 0.8958 - val

In [15]:
model_SPEC.save('../models/mlspec.h5')

In [17]:
model_SPEC.evaluate(test_dataset)

1/1 [==============================] - 2s 2s/step - loss: 1.3220 - mean_absolute_error: 0.9853


[1.3219780921936035, 0.98533314]

## STFT_MAG

In [44]:
X_train = np.load('../data/processed/TDC_VoIP_train_STFT_MAG.npy', allow_pickle=True).item()
X_test = np.load('../data/processed/TDC_VoIP_test_STFT_MAG.npy', allow_pickle=True).item()

In [45]:
train_filenames = X_train['filenames']
y_train_scores = X_train['sample_MOS']
y_train_classes = X_train['class_labels']
X_train_features = X_train['features']
input_height = X_train_features.shape[1]

print("Train data")
print("Features shape:", X_train_features.shape)
print("class_labels shape:", y_train_classes.shape)
print("sample_MOS shape:", y_train_scores.shape)

Train data
Features shape: (327, 257, 1241, 1)
class_labels shape: (327,)
sample_MOS shape: (327,)


In [46]:
test_filenames = X_test['filenames']
y_test_scores = X_test['sample_MOS']
y_test_classes = X_test['class_labels']
X_test_features = X_test['features']

print("Test data")
print("Features shape:", X_test_features.shape)
print("class_labels shape:", y_test_classes.shape)
print("sample_MOS shape:", y_test_scores.shape)

Test data
Features shape: (57, 257, 1241, 1)
class_labels shape: (57,)
sample_MOS shape: (57,)


In [50]:
model_MAG = build_model((input_height, None, 1))

optimizer = tf.keras.optimizers.Adam(LEARNING_RATE)
model_MAG.compile(loss={'output_mos': 'mse'}, 
              loss_weights={'output_mos': 1},
              metrics={'output_mos': tf.keras.metrics.MeanAbsoluteError()},
              optimizer=optimizer)

# Display model architecture summary 
model_MAG.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 257, None, 1)]    0         
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 257, None, 16)     160       
_________________________________________________________________
batch_normalization_22 (Batc (None, 257, None, 16)     64        
_________________________________________________________________
activation_22 (Activation)   (None, 257, None, 16)     0         
_________________________________________________________________
max_pooling2d_22 (MaxPooling (None, 128, None, 16)     0         
_________________________________________________________________
conv2d_23 (Conv2D)           (None, 128, None, 24)     3480      
_________________________________________________________________
batch_normalization_23 (Batc (None, 128, None, 24)     96  

In [51]:
history = model_MAG.fit(X_train_features,
          {'output_mos': y_train_scores},
          validation_data=(X_test_features, 
                           {'output_mos': y_test_scores}),
          batch_size=BATCH_SIZE,
          epochs=1000,
          callbacks=[early_stopping_callback]
        )

Train on 327 samples, validate on 57 samples
Epoch 1/1000
327/327 [==============================] - 110s 337ms/sample - loss: 17.5688 - mean_absolute_error: 4.0009 - val_loss: 12.5991 - val_mean_absolute_error: 3.4452
Epoch 2/1000
327/327 [==============================] - 111s 340ms/sample - loss: 14.9071 - mean_absolute_error: 3.6636 - val_loss: 13.7977 - val_mean_absolute_error: 3.6132
Epoch 3/1000
327/327 [==============================] - 112s 341ms/sample - loss: 13.1068 - mean_absolute_error: 3.4102 - val_loss: 14.0697 - val_mean_absolute_error: 3.6484
Epoch 4/1000
327/327 [==============================] - 111s 338ms/sample - loss: 11.9976 - mean_absolute_error: 3.2293 - val_loss: 13.5882 - val_mean_absolute_error: 3.5781
Epoch 5/1000
327/327 [==============================] - 112s 341ms/sample - loss: 10.7047 - mean_absolute_error: 3.0232 - val_loss: 12.6098 - val_mean_absolute_error: 3.4344
Epoch 6/1000
327/327 [==============================] - 112s 342ms/sample - loss: 9.9

327/327 [==============================] - 114s 347ms/sample - loss: 1.6863 - mean_absolute_error: 1.0348 - val_loss: 2.1992 - val_mean_absolute_error: 1.2917
Epoch 49/1000
327/327 [==============================] - 113s 344ms/sample - loss: 1.6565 - mean_absolute_error: 1.0611 - val_loss: 1.6017 - val_mean_absolute_error: 1.0822
Epoch 50/1000
327/327 [==============================] - 111s 339ms/sample - loss: 1.5684 - mean_absolute_error: 1.0202 - val_loss: 2.1272 - val_mean_absolute_error: 1.2669
Epoch 51/1000
327/327 [==============================] - 111s 339ms/sample - loss: 1.5663 - mean_absolute_error: 1.0126 - val_loss: 1.0680 - val_mean_absolute_error: 0.8755
Epoch 52/1000
327/327 [==============================] - 112s 341ms/sample - loss: 1.3837 - mean_absolute_error: 0.9340 - val_loss: 1.3539 - val_mean_absolute_error: 0.9997
Epoch 53/1000
327/327 [==============================] - 111s 340ms/sample - loss: 1.6388 - mean_absolute_error: 1.0231 - val_loss: 1.7514 - val_mean

327/327 [==============================] - 111s 339ms/sample - loss: 0.7675 - mean_absolute_error: 0.6898 - val_loss: 0.4464 - val_mean_absolute_error: 0.5504
Epoch 143/1000
327/327 [==============================] - 110s 338ms/sample - loss: 0.8923 - mean_absolute_error: 0.7549 - val_loss: 0.5858 - val_mean_absolute_error: 0.6281
Epoch 144/1000
327/327 [==============================] - 111s 340ms/sample - loss: 0.9452 - mean_absolute_error: 0.7774 - val_loss: 0.4400 - val_mean_absolute_error: 0.5468
Epoch 145/1000
327/327 [==============================] - 114s 347ms/sample - loss: 0.8853 - mean_absolute_error: 0.7464 - val_loss: 0.5076 - val_mean_absolute_error: 0.5871
Epoch 146/1000
327/327 [==============================] - 113s 345ms/sample - loss: 0.8261 - mean_absolute_error: 0.7212 - val_loss: 0.5581 - val_mean_absolute_error: 0.6111
Epoch 147/1000
327/327 [==============================] - 111s 339ms/sample - loss: 0.8124 - mean_absolute_error: 0.7349 - val_loss: 0.5433 - val

327/327 [==============================] - 113s 345ms/sample - loss: 0.7077 - mean_absolute_error: 0.6724 - val_loss: 0.3593 - val_mean_absolute_error: 0.4768
Epoch 237/1000
327/327 [==============================] - 114s 349ms/sample - loss: 0.5968 - mean_absolute_error: 0.6171 - val_loss: 0.3782 - val_mean_absolute_error: 0.4765
Epoch 238/1000
327/327 [==============================] - 115s 353ms/sample - loss: 0.6911 - mean_absolute_error: 0.6515 - val_loss: 0.3961 - val_mean_absolute_error: 0.4945
Epoch 239/1000
327/327 [==============================] - 114s 349ms/sample - loss: 0.7170 - mean_absolute_error: 0.6888 - val_loss: 0.6453 - val_mean_absolute_error: 0.6431
Epoch 240/1000
327/327 [==============================] - 114s 348ms/sample - loss: 0.6635 - mean_absolute_error: 0.6515 - val_loss: 0.4358 - val_mean_absolute_error: 0.5292
Epoch 241/1000
327/327 [==============================] - 114s 350ms/sample - loss: 0.6672 - mean_absolute_error: 0.6296 - val_loss: 0.4722 - val

327/327 [==============================] - 111s 339ms/sample - loss: 0.5278 - mean_absolute_error: 0.5747 - val_loss: 0.5928 - val_mean_absolute_error: 0.6151
Epoch 331/1000
327/327 [==============================] - 111s 341ms/sample - loss: 0.5589 - mean_absolute_error: 0.5979 - val_loss: 0.4361 - val_mean_absolute_error: 0.5214
Epoch 332/1000
327/327 [==============================] - 111s 338ms/sample - loss: 0.5386 - mean_absolute_error: 0.5803 - val_loss: 0.3887 - val_mean_absolute_error: 0.4953
Epoch 333/1000
327/327 [==============================] - 111s 338ms/sample - loss: 0.5680 - mean_absolute_error: 0.6007 - val_loss: 0.7914 - val_mean_absolute_error: 0.7199
Epoch 334/1000
327/327 [==============================] - 113s 347ms/sample - loss: 0.6010 - mean_absolute_error: 0.6208 - val_loss: 0.3716 - val_mean_absolute_error: 0.4830
Epoch 335/1000
327/327 [==============================] - 111s 340ms/sample - loss: 0.5925 - mean_absolute_error: 0.6178 - val_loss: 0.4312 - val

327/327 [==============================] - 111s 340ms/sample - loss: 0.4649 - mean_absolute_error: 0.5532 - val_loss: 0.3728 - val_mean_absolute_error: 0.4610
Epoch 425/1000
327/327 [==============================] - 113s 346ms/sample - loss: 0.4501 - mean_absolute_error: 0.5386 - val_loss: 0.3409 - val_mean_absolute_error: 0.4538
Epoch 426/1000
327/327 [==============================] - 111s 341ms/sample - loss: 0.4693 - mean_absolute_error: 0.5289 - val_loss: 0.3530 - val_mean_absolute_error: 0.4545
Epoch 427/1000
327/327 [==============================] - 111s 339ms/sample - loss: 0.4895 - mean_absolute_error: 0.5581 - val_loss: 0.4386 - val_mean_absolute_error: 0.5220
Epoch 428/1000
327/327 [==============================] - 111s 340ms/sample - loss: 0.4792 - mean_absolute_error: 0.5456 - val_loss: 0.3338 - val_mean_absolute_error: 0.4532
Epoch 429/1000
327/327 [==============================] - 111s 340ms/sample - loss: 0.4707 - mean_absolute_error: 0.5516 - val_loss: 0.3471 - val

327/327 [==============================] - 111s 340ms/sample - loss: 0.3437 - mean_absolute_error: 0.4658 - val_loss: 0.3301 - val_mean_absolute_error: 0.4364
Epoch 519/1000
327/327 [==============================] - 111s 339ms/sample - loss: 0.4438 - mean_absolute_error: 0.5419 - val_loss: 0.4560 - val_mean_absolute_error: 0.5093
Epoch 520/1000
327/327 [==============================] - 112s 342ms/sample - loss: 0.3773 - mean_absolute_error: 0.4881 - val_loss: 0.3354 - val_mean_absolute_error: 0.4439
Epoch 521/1000
327/327 [==============================] - 111s 339ms/sample - loss: 0.4139 - mean_absolute_error: 0.5047 - val_loss: 0.5229 - val_mean_absolute_error: 0.5519
Epoch 522/1000
327/327 [==============================] - 111s 341ms/sample - loss: 0.4367 - mean_absolute_error: 0.5318 - val_loss: 0.3328 - val_mean_absolute_error: 0.4403
Epoch 523/1000
327/327 [==============================] - 111s 339ms/sample - loss: 0.3899 - mean_absolute_error: 0.5029 - val_loss: 0.4237 - val

327/327 [==============================] - 111s 341ms/sample - loss: 0.3332 - mean_absolute_error: 0.4658 - val_loss: 0.4012 - val_mean_absolute_error: 0.4831
Epoch 613/1000
327/327 [==============================] - 111s 340ms/sample - loss: 0.3475 - mean_absolute_error: 0.4648 - val_loss: 0.3456 - val_mean_absolute_error: 0.4511
Epoch 614/1000
327/327 [==============================] - 111s 341ms/sample - loss: 0.3719 - mean_absolute_error: 0.4911 - val_loss: 0.4086 - val_mean_absolute_error: 0.4900
Epoch 615/1000
327/327 [==============================] - 111s 339ms/sample - loss: 0.4297 - mean_absolute_error: 0.5192 - val_loss: 0.3581 - val_mean_absolute_error: 0.4576
Epoch 616/1000
327/327 [==============================] - 112s 342ms/sample - loss: 0.3697 - mean_absolute_error: 0.4758 - val_loss: 0.3945 - val_mean_absolute_error: 0.4735
Epoch 617/1000
327/327 [==============================] - 112s 343ms/sample - loss: 0.3377 - mean_absolute_error: 0.4576 - val_loss: 0.3915 - val

In [67]:
result = pd.DataFrame({'true score' :y_test_scores.tolist(),'predicted score': model_MAG.predict(X_test_features).tolist()})

In [68]:
result.head()

,true score,predicted score
0,3.583333,[3.709881067276001]
1,4.750000,[3.1057536602020264]
2,2.666667,[2.9251439571380615]
3,3.916667,[3.3373584747314453]
4,4.000000,[3.8825602531433105]


In [61]:
result.head()

,true score,predicted score
0,3.583333,3.709881


In [56]:
model_MAG.save('../models/model_mga.h5')

## CQT

In [18]:
X_train = np.load('../data/processed/TDC_VoIP_train_CQT.npy', allow_pickle=True).item()
X_test = np.load('../data/processed/TDC_VoIP_test_CQT.npy', allow_pickle=True).item()

In [19]:
train_filenames = X_train['filenames']
y_train_scores = X_train['sample_MOS']
y_train_classes = X_train['class_labels']
X_train_features = X_train['features']
input_height = X_train_features.shape[1]

print("Train data")
print("Features shape:", X_train_features.shape)
print("class_labels shape:", y_train_classes.shape)
print("sample_MOS shape:", y_train_scores.shape)

Train data
Features shape: (327, 84, 1241, 1)
class_labels shape: (327,)
sample_MOS shape: (327,)


In [20]:
test_filenames = X_test['filenames']
y_test_scores = X_test['sample_MOS']
y_test_classes = X_test['class_labels']
X_test_features = X_test['features']

print("Test data")
print("Features shape:", X_test_features.shape)
print("class_labels shape:", y_test_classes.shape)
print("sample_MOS shape:", y_test_scores.shape)

Test data
Features shape: (57, 84, 1241, 1)
class_labels shape: (57,)
sample_MOS shape: (57,)


In [21]:
model_CQT = build_model((input_height, None, 1))

optimizer = tf.keras.optimizers.Adam(LEARNING_RATE)
model_CQT.compile(loss={'output_mos': 'mse'}, 
              loss_weights={'output_mos': 1},
              metrics={'output_mos': tf.keras.metrics.MeanAbsoluteError()},
              optimizer=optimizer)

# Display model architecture summary 
model_CQT.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 84, None, 1)]     0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 84, None, 16)      160       
_________________________________________________________________
batch_normalization_6 (Batch (None, 84, None, 16)      64        
_________________________________________________________________
activation_6 (Activation)    (None, 84, None, 16)      0         
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 42, None, 16)      0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 42, None, 24)      3480      
_________________________________________________________________
batch_normalization_7 (Batch (None, 42, None, 24)      96  

In [22]:
history = model_CQT.fit(X_train_features,
          {'output_mos': y_train_scores},
          validation_data=(X_test_features, 
                           {'output_mos': y_test_scores}),
          batch_size=BATCH_SIZE,
          epochs=1000,
          callbacks=[early_stopping_callback]
        )

Train on 327 samples, validate on 57 samples
Epoch 1/1000
327/327 [==============================] - 33s 102ms/sample - loss: 6.6641 - mean_absolute_error: 2.2452 - val_loss: 11.1765 - val_mean_absolute_error: 3.2315
Epoch 2/1000
327/327 [==============================] - 36s 109ms/sample - loss: 5.2896 - mean_absolute_error: 1.9567 - val_loss: 10.7025 - val_mean_absolute_error: 3.1578
Epoch 3/1000
327/327 [==============================] - 38s 116ms/sample - loss: 4.4090 - mean_absolute_error: 1.7682 - val_loss: 10.5377 - val_mean_absolute_error: 3.1318
Epoch 4/1000
327/327 [==============================] - 35s 108ms/sample - loss: 3.8794 - mean_absolute_error: 1.6425 - val_loss: 10.0969 - val_mean_absolute_error: 3.0608
Epoch 5/1000
327/327 [==============================] - 35s 108ms/sample - loss: 3.2225 - mean_absolute_error: 1.4760 - val_loss: 9.5191 - val_mean_absolute_error: 2.9653
Epoch 6/1000
327/327 [==============================] - 36s 109ms/sample - loss: 2.9325 - mean_a

327/327 [==============================] - 38s 115ms/sample - loss: 1.5859 - mean_absolute_error: 1.0260 - val_loss: 0.7051 - val_mean_absolute_error: 0.7287
Epoch 49/1000
327/327 [==============================] - 38s 116ms/sample - loss: 1.4939 - mean_absolute_error: 0.9962 - val_loss: 0.7160 - val_mean_absolute_error: 0.7348
Epoch 50/1000
327/327 [==============================] - 38s 115ms/sample - loss: 1.3040 - mean_absolute_error: 0.9184 - val_loss: 0.7235 - val_mean_absolute_error: 0.7371
Epoch 51/1000
327/327 [==============================] - 38s 116ms/sample - loss: 1.5888 - mean_absolute_error: 1.0368 - val_loss: 0.7294 - val_mean_absolute_error: 0.7383
Epoch 52/1000
327/327 [==============================] - 39s 119ms/sample - loss: 1.5638 - mean_absolute_error: 1.0145 - val_loss: 0.7549 - val_mean_absolute_error: 0.7545
Epoch 53/1000
327/327 [==============================] - 38s 115ms/sample - loss: 1.4969 - mean_absolute_error: 0.9894 - val_loss: 0.7144 - val_mean_absol

Epoch 96/1000
327/327 [==============================] - 39s 120ms/sample - loss: 1.2131 - mean_absolute_error: 0.8730 - val_loss: 0.9874 - val_mean_absolute_error: 0.8299
Epoch 97/1000
327/327 [==============================] - 37s 114ms/sample - loss: 1.1786 - mean_absolute_error: 0.8631 - val_loss: 0.8989 - val_mean_absolute_error: 0.7881
Epoch 98/1000
327/327 [==============================] - 37s 114ms/sample - loss: 1.0253 - mean_absolute_error: 0.8090 - val_loss: 1.0657 - val_mean_absolute_error: 0.8557
Epoch 99/1000
327/327 [==============================] - 37s 114ms/sample - loss: 1.1285 - mean_absolute_error: 0.8505 - val_loss: 1.0832 - val_mean_absolute_error: 0.8581
Epoch 100/1000
327/327 [==============================] - 37s 114ms/sample - loss: 1.0658 - mean_absolute_error: 0.8048 - val_loss: 1.0086 - val_mean_absolute_error: 0.8365
Epoch 101/1000
327/327 [==============================] - 37s 114ms/sample - loss: 1.1323 - mean_absolute_error: 0.8548 - val_loss: 1.1480 

327/327 [==============================] - 37s 113ms/sample - loss: 0.9107 - mean_absolute_error: 0.7816 - val_loss: 0.8285 - val_mean_absolute_error: 0.7414
Epoch 144/1000
327/327 [==============================] - 37s 114ms/sample - loss: 0.9235 - mean_absolute_error: 0.7477 - val_loss: 0.7559 - val_mean_absolute_error: 0.7125
Epoch 145/1000
327/327 [==============================] - 37s 114ms/sample - loss: 0.8166 - mean_absolute_error: 0.7343 - val_loss: 0.5280 - val_mean_absolute_error: 0.5967
Epoch 146/1000
327/327 [==============================] - 37s 114ms/sample - loss: 0.8111 - mean_absolute_error: 0.7239 - val_loss: 0.5926 - val_mean_absolute_error: 0.6325
Epoch 147/1000
327/327 [==============================] - 38s 116ms/sample - loss: 0.8952 - mean_absolute_error: 0.7654 - val_loss: 0.6773 - val_mean_absolute_error: 0.6707
Epoch 148/1000
327/327 [==============================] - 37s 114ms/sample - loss: 0.9851 - mean_absolute_error: 0.8054 - val_loss: 1.0689 - val_mean_

327/327 [==============================] - 37s 114ms/sample - loss: 0.7604 - mean_absolute_error: 0.7079 - val_loss: 0.5084 - val_mean_absolute_error: 0.5916
Epoch 191/1000
327/327 [==============================] - 37s 114ms/sample - loss: 0.8341 - mean_absolute_error: 0.7394 - val_loss: 0.9932 - val_mean_absolute_error: 0.8123
Epoch 192/1000
327/327 [==============================] - 37s 114ms/sample - loss: 0.7354 - mean_absolute_error: 0.6842 - val_loss: 0.6616 - val_mean_absolute_error: 0.6579
Epoch 193/1000
327/327 [==============================] - 38s 115ms/sample - loss: 0.7523 - mean_absolute_error: 0.6827 - val_loss: 0.8374 - val_mean_absolute_error: 0.7404
Epoch 194/1000
327/327 [==============================] - 37s 114ms/sample - loss: 0.6767 - mean_absolute_error: 0.6555 - val_loss: 0.7871 - val_mean_absolute_error: 0.7255
Epoch 195/1000
327/327 [==============================] - 37s 114ms/sample - loss: 0.7965 - mean_absolute_error: 0.7226 - val_loss: 0.9721 - val_mean_

...

327/327 [==============================] - 37s 114ms/sample - loss: 0.2787 - mean_absolute_error: 0.4174 - val_loss: 0.4067 - val_mean_absolute_error: 0.5128
Epoch 708/1000
327/327 [==============================] - 37s 114ms/sample - loss: 0.2956 - mean_absolute_error: 0.4362 - val_loss: 0.5495 - val_mean_absolute_error: 0.6019
Epoch 709/1000
327/327 [==============================] - 37s 114ms/sample - loss: 0.3026 - mean_absolute_error: 0.4295 - val_loss: 0.9306 - val_mean_absolute_error: 0.8208
Epoch 710/1000
327/327 [==============================] - 37s 114ms/sample - loss: 0.2724 - mean_absolute_error: 0.4135 - val_loss: 0.4952 - val_mean_absolute_error: 0.5687
Epoch 711/1000
327/327 [==============================] - 37s 114ms/sample - loss: 0.2863 - mean_absolute_error: 0.4163 - val_loss: 0.3143 - val_mean_absolute_error: 0.4404
Epoch 712/1000
327/327 [==============================] - 37s 114ms/sample - loss: 0.3086 - mean_absolute_error: 0.4445 - val_loss: 0.7030 - val_mean_

327/327 [==============================] - 37s 114ms/sample - loss: 0.2776 - mean_absolute_error: 0.4049 - val_loss: 0.2966 - val_mean_absolute_error: 0.4246
Epoch 755/1000
327/327 [==============================] - 37s 114ms/sample - loss: 0.2997 - mean_absolute_error: 0.4362 - val_loss: 0.3718 - val_mean_absolute_error: 0.4804
Epoch 756/1000
327/327 [==============================] - 37s 113ms/sample - loss: 0.3031 - mean_absolute_error: 0.4460 - val_loss: 0.4001 - val_mean_absolute_error: 0.5071
Epoch 757/1000
327/327 [==============================] - 37s 114ms/sample - loss: 0.2799 - mean_absolute_error: 0.4192 - val_loss: 0.2630 - val_mean_absolute_error: 0.4002
Epoch 758/1000
327/327 [==============================] - 37s 114ms/sample - loss: 0.2731 - mean_absolute_error: 0.4124 - val_loss: 0.6553 - val_mean_absolute_error: 0.6852
Epoch 759/1000
327/327 [==============================] - 40s 121ms/sample - loss: 0.2856 - mean_absolute_error: 0.4227 - val_loss: 0.5484 - val_mean_

327/327 [==============================] - 37s 113ms/sample - loss: 0.2554 - mean_absolute_error: 0.3884 - val_loss: 0.2761 - val_mean_absolute_error: 0.3934
Epoch 802/1000
327/327 [==============================] - 37s 114ms/sample - loss: 0.2720 - mean_absolute_error: 0.3993 - val_loss: 0.3962 - val_mean_absolute_error: 0.4738
Epoch 803/1000
327/327 [==============================] - 37s 114ms/sample - loss: 0.2366 - mean_absolute_error: 0.3865 - val_loss: 0.3498 - val_mean_absolute_error: 0.4389
Epoch 804/1000
327/327 [==============================] - 37s 114ms/sample - loss: 0.2669 - mean_absolute_error: 0.4137 - val_loss: 0.2425 - val_mean_absolute_error: 0.3862
Epoch 805/1000
327/327 [==============================] - 37s 114ms/sample - loss: 0.3249 - mean_absolute_error: 0.4478 - val_loss: 0.2493 - val_mean_absolute_error: 0.3825
Epoch 806/1000
327/327 [==============================] - 37s 114ms/sample - loss: 0.2783 - mean_absolute_error: 0.4146 - val_loss: 0.2345 - val_mean_

327/327 [==============================] - 37s 115ms/sample - loss: 0.2520 - mean_absolute_error: 0.4035 - val_loss: 0.5902 - val_mean_absolute_error: 0.6474
Epoch 849/1000
327/327 [==============================] - 38s 115ms/sample - loss: 0.2328 - mean_absolute_error: 0.3751 - val_loss: 0.4705 - val_mean_absolute_error: 0.5526
Epoch 850/1000
327/327 [==============================] - 37s 113ms/sample - loss: 0.2763 - mean_absolute_error: 0.4273 - val_loss: 0.6660 - val_mean_absolute_error: 0.6791
Epoch 851/1000
327/327 [==============================] - 37s 114ms/sample - loss: 0.2369 - mean_absolute_error: 0.3943 - val_loss: 0.5657 - val_mean_absolute_error: 0.6409
Epoch 852/1000
327/327 [==============================] - 37s 113ms/sample - loss: 0.2503 - mean_absolute_error: 0.3871 - val_loss: 0.7506 - val_mean_absolute_error: 0.7484
Epoch 853/1000
327/327 [==============================] - 37s 114ms/sample - loss: 0.2820 - mean_absolute_error: 0.4154 - val_loss: 0.3751 - val_mean_

327/327 [==============================] - 37s 114ms/sample - loss: 0.2410 - mean_absolute_error: 0.3904 - val_loss: 0.6364 - val_mean_absolute_error: 0.6696
Epoch 896/1000
327/327 [==============================] - 37s 114ms/sample - loss: 0.2239 - mean_absolute_error: 0.3705 - val_loss: 0.6465 - val_mean_absolute_error: 0.6731
Epoch 897/1000
327/327 [==============================] - 37s 114ms/sample - loss: 0.2545 - mean_absolute_error: 0.3967 - val_loss: 0.6313 - val_mean_absolute_error: 0.6672
Epoch 898/1000
327/327 [==============================] - 37s 114ms/sample - loss: 0.2863 - mean_absolute_error: 0.4159 - val_loss: 0.3036 - val_mean_absolute_error: 0.4407
Epoch 899/1000
327/327 [==============================] - 39s 119ms/sample - loss: 0.2440 - mean_absolute_error: 0.3817 - val_loss: 0.4170 - val_mean_absolute_error: 0.5040
Epoch 900/1000
327/327 [==============================] - 37s 114ms/sample - loss: 0.2450 - mean_absolute_error: 0.3932 - val_loss: 0.5229 - val_mean_

In [23]:
result = pd.DataFrame({'true score' :y_test_scores.tolist(),'predicted score': model_CQT.predict(X_test_features).tolist()})

In [24]:
result.head()

,true score,predicted score
0,1.875000,[2.013476610183716]
1,3.708333,[3.7875585556030273]
2,4.083333,[4.143774032592773]
3,1.708333,[1.9634042978286743]
4,4.166667,[3.330070972442627]


In [25]:
model_CQT.save('../models/model_cqt.h5')

## MLE SPEC

In [27]:
X_train = np.load('../data/processed/TDC_VoIP_train_MEL_SPEC.npy', allow_pickle=True).item()
X_test = np.load('../data/processed/TDC_VoIP_test_MEL_SPEC.npy', allow_pickle=True).item()

In [28]:
train_filenames = X_train['filenames']
y_train_scores = X_train['sample_MOS']
y_train_classes = X_train['class_labels']
X_train_features = X_train['features']
input_height = X_train_features.shape[1]

print("Train data")
print("Features shape:", X_train_features.shape)
print("class_labels shape:", y_train_classes.shape)
print("sample_MOS shape:", y_train_scores.shape)

Train data
Features shape: (327, 128, 1241, 1)
class_labels shape: (327,)
sample_MOS shape: (327,)


In [29]:
test_filenames = X_test['filenames']
y_test_scores = X_test['sample_MOS']
y_test_classes = X_test['class_labels']
X_test_features = X_test['features']

print("Test data")
print("Features shape:", X_test_features.shape)
print("class_labels shape:", y_test_classes.shape)
print("sample_MOS shape:", y_test_scores.shape)

Test data
Features shape: (57, 128, 1241, 1)
class_labels shape: (57,)
sample_MOS shape: (57,)


In [30]:
model_MLE = build_model((input_height, None, 1))

optimizer = tf.keras.optimizers.Adam(LEARNING_RATE)
model_MLE.compile(loss={'output_mos': 'mse'}, 
              loss_weights={'output_mos': 1},
              metrics={'output_mos': tf.keras.metrics.MeanAbsoluteError()},
              optimizer=optimizer)

# Display model architecture summary 
model_MLE.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 128, None, 1)]    0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 128, None, 16)     160       
_________________________________________________________________
batch_normalization_12 (Batc (None, 128, None, 16)     64        
_________________________________________________________________
activation_12 (Activation)   (None, 128, None, 16)     0         
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 64, None, 16)      0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 64, None, 24)      3480      
_________________________________________________________________
batch_normalization_13 (Batc (None, 64, None, 24)      96  

In [31]:
history = model_MLE.fit(X_train_features,
          {'output_mos': y_train_scores},
          validation_data=(X_test_features, 
                           {'output_mos': y_test_scores}),
          batch_size=BATCH_SIZE,
          epochs=1000,
          callbacks=[early_stopping_callback]
        )

Train on 327 samples, validate on 57 samples
Epoch 1/1000
327/327 [==============================] - 54s 165ms/sample - loss: 3.9122 - mean_absolute_error: 1.6905 - val_loss: 10.2459 - val_mean_absolute_error: 3.0831
Epoch 2/1000
327/327 [==============================] - 55s 167ms/sample - loss: 3.4789 - mean_absolute_error: 1.5656 - val_loss: 10.0592 - val_mean_absolute_error: 3.0520
Epoch 3/1000
327/327 [==============================] - 74s 227ms/sample - loss: 3.2723 - mean_absolute_error: 1.5316 - val_loss: 9.5471 - val_mean_absolute_error: 2.9661
Epoch 4/1000
327/327 [==============================] - 92s 280ms/sample - loss: 2.6930 - mean_absolute_error: 1.3753 - val_loss: 9.0626 - val_mean_absolute_error: 2.8833
Epoch 5/1000
327/327 [==============================] - 92s 281ms/sample - loss: 2.4379 - mean_absolute_error: 1.2973 - val_loss: 8.2052 - val_mean_absolute_error: 2.7296
Epoch 6/1000
327/327 [==============================] - 92s 281ms/sample - loss: 2.4424 - mean_abs

327/327 [==============================] - 89s 272ms/sample - loss: 1.3905 - mean_absolute_error: 0.9807 - val_loss: 0.8630 - val_mean_absolute_error: 0.8190
Epoch 49/1000
327/327 [==============================] - 90s 274ms/sample - loss: 1.2502 - mean_absolute_error: 0.9035 - val_loss: 0.8261 - val_mean_absolute_error: 0.7989
Epoch 50/1000
327/327 [==============================] - 90s 277ms/sample - loss: 1.2681 - mean_absolute_error: 0.9269 - val_loss: 0.8163 - val_mean_absolute_error: 0.7937
Epoch 51/1000
327/327 [==============================] - 90s 276ms/sample - loss: 1.1808 - mean_absolute_error: 0.9000 - val_loss: 0.7945 - val_mean_absolute_error: 0.7815
Epoch 52/1000
327/327 [==============================] - 91s 278ms/sample - loss: 1.2102 - mean_absolute_error: 0.9119 - val_loss: 0.8051 - val_mean_absolute_error: 0.7878
Epoch 53/1000
327/327 [==============================] - 91s 279ms/sample - loss: 1.4477 - mean_absolute_error: 0.9971 - val_loss: 0.8907 - val_mean_absol

Epoch 96/1000
327/327 [==============================] - 56s 171ms/sample - loss: 1.1541 - mean_absolute_error: 0.8850 - val_loss: 0.8023 - val_mean_absolute_error: 0.7848
Epoch 97/1000
327/327 [==============================] - 56s 171ms/sample - loss: 1.2842 - mean_absolute_error: 0.9368 - val_loss: 0.7880 - val_mean_absolute_error: 0.7776
Epoch 98/1000
327/327 [==============================] - 56s 171ms/sample - loss: 1.0951 - mean_absolute_error: 0.8728 - val_loss: 0.7830 - val_mean_absolute_error: 0.7749
Epoch 99/1000
327/327 [==============================] - 56s 172ms/sample - loss: 1.1846 - mean_absolute_error: 0.9034 - val_loss: 0.7830 - val_mean_absolute_error: 0.7752
Epoch 100/1000
327/327 [==============================] - 56s 172ms/sample - loss: 1.3026 - mean_absolute_error: 0.9488 - val_loss: 0.7865 - val_mean_absolute_error: 0.7770
Epoch 101/1000
327/327 [==============================] - 56s 171ms/sample - loss: 1.2262 - mean_absolute_error: 0.9167 - val_loss: 0.8008 

327/327 [==============================] - 56s 172ms/sample - loss: 1.1043 - mean_absolute_error: 0.8715 - val_loss: 0.7943 - val_mean_absolute_error: 0.7834
Epoch 144/1000
327/327 [==============================] - 56s 171ms/sample - loss: 1.1545 - mean_absolute_error: 0.8961 - val_loss: 0.8007 - val_mean_absolute_error: 0.7856
Epoch 145/1000
327/327 [==============================] - 56s 171ms/sample - loss: 1.1144 - mean_absolute_error: 0.8632 - val_loss: 0.8064 - val_mean_absolute_error: 0.7888
Epoch 146/1000
327/327 [==============================] - 56s 171ms/sample - loss: 1.0566 - mean_absolute_error: 0.8435 - val_loss: 0.7844 - val_mean_absolute_error: 0.7760
Epoch 147/1000
327/327 [==============================] - 56s 171ms/sample - loss: 1.0608 - mean_absolute_error: 0.8565 - val_loss: 0.7771 - val_mean_absolute_error: 0.7733
Epoch 148/1000
327/327 [==============================] - 56s 172ms/sample - loss: 1.0893 - mean_absolute_error: 0.8732 - val_loss: 0.8113 - val_mean_

327/327 [==============================] - 56s 172ms/sample - loss: 1.0134 - mean_absolute_error: 0.8104 - val_loss: 0.7877 - val_mean_absolute_error: 0.7816
Epoch 191/1000
327/327 [==============================] - 56s 171ms/sample - loss: 0.9790 - mean_absolute_error: 0.7950 - val_loss: 0.7619 - val_mean_absolute_error: 0.7668
Epoch 192/1000
327/327 [==============================] - 56s 173ms/sample - loss: 1.0375 - mean_absolute_error: 0.8225 - val_loss: 0.8591 - val_mean_absolute_error: 0.8161
Epoch 193/1000
327/327 [==============================] - 56s 171ms/sample - loss: 1.0784 - mean_absolute_error: 0.8740 - val_loss: 0.8274 - val_mean_absolute_error: 0.7991
Epoch 194/1000
327/327 [==============================] - 56s 171ms/sample - loss: 0.9785 - mean_absolute_error: 0.8201 - val_loss: 0.8147 - val_mean_absolute_error: 0.7948
Epoch 195/1000
327/327 [==============================] - 56s 173ms/sample - loss: 1.0336 - mean_absolute_error: 0.8230 - val_loss: 0.8303 - val_mean_

327/327 [==============================] - 56s 171ms/sample - loss: 0.8419 - mean_absolute_error: 0.7634 - val_loss: 0.8308 - val_mean_absolute_error: 0.7976
Epoch 238/1000
327/327 [==============================] - 57s 173ms/sample - loss: 0.9259 - mean_absolute_error: 0.7885 - val_loss: 0.9188 - val_mean_absolute_error: 0.8351
Epoch 239/1000
327/327 [==============================] - 56s 172ms/sample - loss: 0.9007 - mean_absolute_error: 0.7791 - val_loss: 0.9402 - val_mean_absolute_error: 0.8442
Epoch 240/1000
327/327 [==============================] - 56s 172ms/sample - loss: 0.9394 - mean_absolute_error: 0.7936 - val_loss: 0.9312 - val_mean_absolute_error: 0.8404
Epoch 241/1000
327/327 [==============================] - 56s 172ms/sample - loss: 0.8957 - mean_absolute_error: 0.7776 - val_loss: 1.0427 - val_mean_absolute_error: 0.8854
Epoch 242/1000
327/327 [==============================] - 56s 171ms/sample - loss: 0.9244 - mean_absolute_error: 0.7980 - val_loss: 0.9254 - val_mean_

In [32]:
result = pd.DataFrame({'true score' :y_test_scores.tolist(),'predicted score': model_MLE.predict(X_test_features).tolist()})

In [33]:
result.head()

,true score,predicted score
0,3.583333,[3.2805795669555664]
1,4.750000,[2.975260019302368]
2,2.666667,[3.4010071754455566]
3,3.916667,[2.9504153728485107]
4,4.000000,[3.1192195415496826]


In [34]:
model_MLE.save('../models/model_mle.h5')